In [ ]:
import numpy as np
import math
import sys
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
#from ml_from_scratch.utils import normalize, standardize, polynomial_features

# Linear Regression

In [ ]:
def main():
    n_samples = 100
    n_features = 2
    test_size = 0.25
    n_iterations = 100
    learning_rate = 0.001

    # Create dataset
    X, y = make_regression(n_samples=n_samples, n_features=n_features, noise=0)

    # Create train, test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    #print(X_train.shape)

    model = LinearRegression(n_iterations=n_iterations, learning_rate=learning_rate, gradient_descent=True)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(mean_squared_error(y_pred, y_test))

    # Training error plot
    n = len(model.training_errors)
    training, = plt.plot(range(n), model.training_errors, label="Training Error")
    plt.legend(handles=[training])
    plt.title("Error Plot")
    plt.ylabel('Mean Squared Error')
    plt.xlabel('Iterations')
    plt.show()

if __name__ == "__main__":
    main()

In [ ]:
class Regression(object):
    """
    Models relationship b/w  independent variables X and dependent variable y
    Base class for linear regression models
    Parameters
    ----------
    n_iterations: float
        Number of training iterations
    learning_rate: float
        Step size of optimisation algorithm
    """

    def __init__(self, n_iterations, learning_rate):
        self.n_iterations = n_iterations
        self.learning_rate = learning_rate
    
    def initialize_weights(self, n_features):
        """Initialise weights randomly b/w [-1/sqrt(N) 1/sqrt(N)]"""
        limit = 1 / math.sqrt(n_features)
        self.w = np.random.uniform(-limit, limit, (n_features, ))
    
    def fit(self, X, y):
        """Learn parameters using supervised data"""
        # Insert column of 1s for bias term
        X = np.insert(X, 0, 1, axis=1)
        # Initialise weights
        self.initialize_weights(n_features=X.shape[1])
        self.training_errors = []

        # Gradient descent for n_iterations
        for i in range(self.n_iterations):
            # Predicted output
            y_pred = X.dot(self.w)
            # Error (target - predicted)
            # L2 Loss
            mse = np.mean(0.5*(y - y_pred)**2) + self.regularization(self.w)
            self.training_errors.append(mse)
            # Gradient of error
            grad_w = -(y - y_pred).dot(X) + self.regularization.grad(self.w)
            # Gradient descent step
            self.w -= self.learning_rate * grad_w
    
    def predict(self, X):
        """Predictions for new data"""
        X = np.insert(X, 0, 1, axis=1)
        y_pred = X.dot(self.w)
        return y_pred

In [ ]:
class LinearRegression(Regression):
    """Linear Regression Model
    """
    def __init__(self, n_iterations=100, learning_rate=0.0001, gradient_descent=True):
        self.gradient_descent = gradient_descent
        # No regularisation
        self.regularization = lambda x: 0
        self.regularization.grad = lambda x: 0
        super(LinearRegression, self).__init__(n_iterations=n_iterations, learning_rate=learning_rate)
    
    def fit(self, X, y):
        """
        If gradient descent use base class's fit
        If not gradient descent use normal equations (inverse using SVD)
        """
        if self.gradient_descent:
            super(LinearRegression, self).fit(X, y)
        else:
            X = np.insert(X, 0, 1, axis=1)
            # Moore-Penrose pseudo inverse
            U, S, V = np.linalg.svd(X.T.dot(X))
            S = np.diag(S)
            Xtx_inv = V.dot(np.linalg.pinv(S)).dot(U.T)
            self.w = Xtx_inv.dot(X.T).dot(y)
            self.training_errors = []